# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

In [6]:
# parse the xml, and to make things cleaner, just assign the root to a variable

document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

In [7]:
# Part 1 - 10 countries with the lowest infant mortality rates
# Create some placeholder data frames to host the country names and infant mortality rates

country_name = []
infant_mortality = []

# Now we iterate through the XML to put the data into the data frames
for element in root:
    for subelement in element.getiterator('infant_mortality'):
        country_name.append(element.find('name').text)
        infant_mortality.append(np.float(subelement.text))
        
# Here we're just creating the definition of the dataframe that we want, which are the two columns combined
definition = {'country_name':country_name, 'infant_mortality':infant_mortality}
bottom10mortality = pd.DataFrame(data=definition)

# Now sort by infant mortality rate and take the first 10
bottom10mortality.sort_values('infant_mortality', ascending=True).head(10)


,country_name,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [8]:
# Part 2 - 10 cities with the largest population

#so this is trickier because now instead of a 1 : 1 relationship like part 1, a city may have one or more population
#observations. Which means I now have to figure out how to get that to happen.

#and sometimes of course, a city country might have more than one name attribute. Ugh.

city_name = []
population_number = []
population_type = []
year = []

# so the city is a child of the root, but the population is a grandchild of the root. So we'll have to nest some things together
# the assignment isn't clear which population measure we should be taking, so I'll include the measurement type as well
# note that not all of the population measurements have a "measured" type

for element in root:
    for city in element.getiterator('city'):
        for population in city.getiterator('population'):
            city_name.append(city.find('name').text)
            population_number.append(np.int(population.text))
            year.append(np.int(population.get('year')))
            if 'measured' in population.attrib:
                population_type.append(population.get('measured'))
            else:
                population_type.append(np.nan)
                

# now define the end result data frame

definition = {'city_name':city_name, 'year':year, 'population_type':population_type,'population':population_number}
citypopulation = pd.DataFrame(data=definition)

# and pull out the 10 cities with the largest population

citypopulation.loc[citypopulation.reset_index().groupby('city_name')['year'].idxmax()]. \
sort_values('population', ascending=False).head(10)

,city_name,population,population_type,year
3750,Shanghai,22315474,census,2010
2607,Istanbul,13710512,admin.,2012
4303,Mumbai,12442373,census,2011
1546,Moskva,11979529,estimate,2013
3746,Beijing,11716620,census,2010
8208,São Paulo,11152344,census,2010
3754,Tianjin,11090314,census,2010
3364,Guangzhou,11071424,census,2010
4399,Delhi,11034555,census,2011
3371,Shenzhen,10358381,census,2010


In [9]:
# Part 3 - 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

# ethnic group is a child of the root. Can have multiples per country
# Population is also a child of the root - and it looks like we need measured = est.
# I'm guessing we'll have to multiply the ethnicgroup percentage by the population of the country
# to get the population of the ethnic group

ethnic_population = []
ethnic_group = []
year = []
country_name = []

# so we're going to get the country, ethnic group, year, and calculate the population which is percent * population / 100
for element in root:
    for country in element.getiterator('country'):
        for population in country.findall('population'):
            if country.find('ethnicgroup') is not None:
                country_name.append(element.find('name').text)
                ethnic_group.append(element.find('ethnicgroup').text)
                year.append(population.get('year'))
                ethnic_population.append(np.float(np.int(population.text) * np.float(element.find('ethnicgroup'). \
                                                                         get('percentage'))/100))
            
# create the data frame
definition = {'ethnic_group':ethnic_group, 'country':country_name, 'year':year, 'ethnic_population':ethnic_population}
ethnic_pop = pd.DataFrame(data=definition)

# get the maximum country/year combination to get the best/latest estimate of population
ethnic_pop = ethnic_pop.loc[ethnic_pop.reset_index().groupby(['ethnic_group', 'country'])['year'].idxmax()]

# and now return the top 10 list of ethnic groups by population
ethnic_pop.groupby(['ethnic_group']).agg(np.sum).sort_values('ethnic_population', ascending=False).head(10)

,ethnic_population
ethnic_group,
Han Chinese,1.245059e+09
European,4.410033e+08
Dravidian,3.027137e+08
African,1.986050e+08
Bengali,1.467769e+08
Mestizo,1.419729e+08
Japanese,1.265342e+08
Russian,1.146462e+08
Javanese,1.134560e+08


In [10]:
# Part 4 - name and country of a) longest river, b) largest lake and c) airport at highest elevation

# the problem with this one is that the river, lake, airport have country codes associated with them
# the countries also have country codes. the river/lake/airport data are not children of the countries
# so this implies some kind of lookup to get the country name from the country code

# create a dataframe that will hold the country codes and country names - we'll use this for all 3 parts

country_code = []
country_name = []

for element in root:
    for country in element.getiterator('country'):
        country_code.append(country.get('car_code'))
        country_name.append(country.find('name').text)

definition = {'country_code':country_code,
             'country_name':country_name}

country = pd.DataFrame(data=definition)


In [11]:
#a) longest river

# There seems to be multiple countries on any given river - whether you look at the source country
# or just the country. So we'll have to retrieve both names, and we'll assume that we're using the source country field.

river_name = []
river_length = []
river_country = []

for element in root:
    for river in element.getiterator('river'):
        river_name.append(river.find('name').text)
        if river.find('length') is not None:
            river_length.append(np.float(river.find('length').text))
        else:
            river_length.append(np.nan)
        river_country.append(river.find('source').get('country'))
#        print(river.find('name').text, river.find('length').text, river.get('country'))

# define and make the data frame
definition = {'river_name':river_name, 'river_length':river_length, 'country_code':river_country}
longest_river = pd.DataFrame(data=definition)

longest_river = longest_river.sort_values('river_length', ascending=False).head(1)

pd.merge(country, longest_river, on='country_code')


,country_code,country_name,river_length,river_name
0,PE,Peru,6448,Amazonas


In [12]:
# b) largest lake

lake_name = []
lake_size = []
lake_country = []

for element in root:
    for lake in element.iter('lake'):
        lake_name.append(lake.find('name').text)
        if lake.find('area') is not None:
            lake_size.append(np.float(lake.find('area').text))
        else:
            lake_size.append(np.nan)
        if lake.find('located') is not None:
            lake_country.append(lake.find('located').get('country'))
        else:
            lake_country.append(np.nan)
        
definition = {'lake_name':lake_name,
             'lake_size':lake_size,
             'country_code':lake_country}

largest_lake = pd.DataFrame(data=definition)

largest_lake = largest_lake.sort_values('lake_size',ascending=False).head(1)

pd.merge(country, largest_lake, on='country_code')

,country_code,country_name,lake_name,lake_size
0,R,Russia,Caspian Sea,386400


In [13]:
# c) airport at highest elevation

airport_name = []
airport_country = []
airport_elevation = []

for element in root:
    for airport in element.getiterator('airport'):
        airport_name.append(airport.find('name').text)
        airport_country.append(airport.get('country'))
        if airport.find('elevation') is not None:
            if airport.find('elevation').text is not None:
                airport_elevation.append(np.float(airport.find('elevation').text))
            else:
                airport_elevation.append(0)

definition = {'airport_name':airport_name,
             'country_code':airport_country,
             'airport_elevation':airport_elevation}

highest_airport = pd.DataFrame(data=definition)

highest_airport = highest_airport.sort_values('airport_elevation', ascending=False).head(1)

pd.merge(country, highest_airport, on='country_code')


,country_code,country_name,airport_elevation,airport_name
0,BOL,Bolivia,4063,El Alto Intl
